In [1]:
import pandas as pd
import random
import numpy as np
import string
import re
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split

In [2]:
datasets_path  = 'D:/code_lab/twitter_sentiment_analys_training/datasets'

In [71]:
df = pd.read_csv(f'{datasets_path}/tweet.csv')
del df[df.columns[0]]
df

,sentimen,tweet
0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...
1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib..."
2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik...."
3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p..."
4,negatif,Negara kita ngutang buat bngun infrastruktur y...
...,...,...
1810,netral,Negarawan sejati sll bangga dan mengedepankan ...
1811,netral,1. HRS ceramah di Damai Indonesiaku 2. Perekon...
1812,netral,Mari bangun bangsa dgn mendukung perekonomian ...
1813,netral,Bantu majukan perekonomian bangsa bersama Pak ...


In [72]:
df.sentimen.value_counts()

sentimen
positif    612
netral     607
negatif    596
Name: count, dtype: int64

# Cleaning

In [73]:
tweets_cleaned = []

  # Looping setiap kalimat
for sentence in df.tweet:

    sentence = re.sub(r"(pic.twitter.com/[^\s]+)", '', sentence) # remove twitter image link
  
    sentence = re.sub(r'http[s]?\://\S+', "", sentence) # remove url

    sentence = re.sub(r'\S*.com.*\S', "", sentence)

    sentence = re.sub(r'\S*.co.id.*\S', "", sentence)

    sentence = re.sub(r'([a-z])([A-Z])', r'\1 \2', sentence) # remove kata kata berdempet

    sentence = re.sub(r"#", "", sentence) #remove hashtag

    sentence = sentence.replace('@', '') # remove at@

    sentence = re.sub(r'\b\d+\b', '', sentence) # remove angka berdempet dg kata

    sentence = re.sub('\d', '', sentence) # remove angka

    sentence = re.sub(r'\b(\w+)\s+\1\b', r'\1', sentence) # Remove reapeating char

    sentence = re.sub(r'[^a-zA-Z0-9\s]', '', sentence) # remove punctuation and special character

    sentence = re.sub(r"\s+", " ", sentence) # remove double space

    sentence = sentence.lower() # case folding

    sentence = sentence.strip()

    tweets_cleaned.append(sentence)

In [76]:
display(df.tweet[8])
display(tweets_cleaned[8])

'Iya bener Aa, kita MANTAP kan pilihan ke Pemimpin yg bs memberi Solusi @jokowi bukan yg menyalahkan pemimpin sebelumnya @SBYudhoyono dismash sm mas Bowo, percuma koalisi klau mengantam tmn koalisi ðŸ˜‚ & yg plg penting pilih Pemimpin yg Rajin Ibadah Wajib, Bisa NGAJI & JumatanðŸ™\x8fðŸ‘\x8d'

'iya bener aa kita mantap kan pilihan ke pemimpin yg bs memberi solusi jokowi bukan yg menyalahkan pemimpin sebelumnya sbyudhoyono dismash sm mas bowo percuma koalisi klau mengantam tmn koalisi yg plg penting pilih pemimpin yg rajin ibadah wajib bisa ngaji jumatan'

# Spell Correcting

In [77]:
spell_dictionary = open(f'{datasets_path}/slangword.txt').read()
spell_dictionary

'&:dan\n+:tambah\n/:atau\n=:sama dengan\nababil:anak ingusan\nabal2:palsu\nabal:palsu\nad:ada\nakooh:aku\nalay:norak\nalbm:album\nampe:sampai\nanjir:waw\nanyway:ngomong-ngomong\naq:aku\nasap:secepatnya\nato:atau\natw:atau\nava:foto profil\nbaget:keras kepala\nbaper:bawa perasaan\nbapuk:rusak\nbcr:bicara\nbebeb:pacar\nbegin:awal\nbejibun:bertumpuk banyak\nbener:benar\nber2:berdua\nber3:bertiga\nbetter:lebih baik\nbf:pacar\nbgt:banget\nbhas:bahas\nbhg:bahagia\nbikin:buat\nbinggo:banget\nbingit:banget\nbklan:bakalan\nbkn:bukan\nblg:bilang\nbnr:benar\nbnyk:banyak\nbr:baru\nbrb:akan segera kembali\nbrjaya:berjaya\nbs:bisa\nbsk:besok\nbtw:ngomong-ngomong\ncm:cuma\ncmiiw:koreksi saya\ncnt:cinta\ncongrats:selamat\ncrop:potong\ncya:sampai jumpa lagi\nd:di\nda:ada\ndafuk:kurang ajar\ndamat:tidak peduli\ndempa:demi apa\ndgn:dengan\ndkt:dekat\ndlm:dalam\ndlu:dulu\ndmn:dimana\ndpt:dapat\ndr:dari\ndrpd:daripada\ndsna:disana\ndugem:dunia gemerlap\nelo:kamu\nelu:kamu\nemg:memang\nepsd:episode\nfake:fa

In [78]:
def create_spell_check_dictionary(spell_check_data):
    spell_check_dict = {}
    lines = spell_check_data.strip().split("\n")
    for line in lines:
        key, value = line.split(":")
        spell_check_dict[key] = value
    return spell_check_dict

In [79]:
spell_dictionary = create_spell_check_dictionary(spell_dictionary)
spell_dictionary

{'&': 'dan',
 '+': 'tambah',
 '/': 'atau',
 '=': 'sama dengan',
 'ababil': 'anak ingusan',
 'abal2': 'palsu',
 'abal': 'palsu',
 'ad': 'ada',
 'akooh': 'aku',
 'alay': 'norak',
 'albm': 'album',
 'ampe': 'sampai',
 'anjir': 'waw',
 'anyway': 'ngomong-ngomong',
 'aq': 'aku',
 'asap': 'secepatnya',
 'ato': 'atau',
 'atw': 'atau',
 'ava': 'foto profil',
 'baget': 'keras kepala',
 'baper': 'bawa perasaan',
 'bapuk': 'rusak',
 'bcr': 'bicara',
 'bebeb': 'pacar',
 'begin': 'awal',
 'bejibun': 'bertumpuk banyak',
 'bener': 'benar',
 'ber2': 'berdua',
 'ber3': 'bertiga',
 'better': 'lebih baik',
 'bf': 'pacar',
 'bgt': 'banget',
 'bhas': 'bahas',
 'bhg': 'bahagia',
 'bikin': 'buat',
 'binggo': 'banget',
 'bingit': 'banget',
 'bklan': 'bakalan',
 'bkn': 'bukan',
 'blg': 'bilang',
 'bnr': 'benar',
 'bnyk': 'banyak',
 'br': 'baru',
 'brb': 'akan segera kembali',
 'brjaya': 'berjaya',
 'bs': 'bisa',
 'bsk': 'besok',
 'btw': 'ngomong-ngomong',
 'cm': 'cuma',
 'cmiiw': 'koreksi saya',
 'cnt': 'cinta

In [80]:
def tweet_spell_correction(tweet):
    words = tweet.split()

    corrected_words = []

    for word in words:
        if word in spell_dictionary:
            corrected_word = spell_dictionary[word]
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    corrected_tweet = ' '.join(corrected_words)

    return corrected_tweet

In [81]:
tweet_spell_corrected = []

for idx in range(len(tweets_cleaned)):
    corrected = tweet_spell_correction(tweets_cleaned[idx])
    tweet_spell_corrected.append(corrected)

display(tweets_cleaned[:5])
print('--'*50)
display(tweet_spell_corrected[:5])

['kata prabowo indonesia tidak dihargai bangsa asing berita ini pasti hoax buatan penguasa ya kan rockygerung',
 'batuan langka tasbih jokowi hadiah dari habib luthfi seharga mercy',
 'di era jokowi ekonomi indonesia semakin baik indonesia maju jokowi lagi jokowi menang total debat',
 'bagi sumatera selatan asian games berdampak pd ekonomi langsung diprediksi mencapai triliun indonesia maju jokowi hebat',
 'negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat terus masyarakatnya ngeluh karena negara ngutang setiap negara itu pasti ngutang utang bisa dibayar kalo negara dapet penghasilan penghasilan negara itu ya dari pajak']

----------------------------------------------------------------------------------------------------


['kata prabowo indonesia tidak dihargai bangsa asing berita ini pasti palsu buatan penguasa ya kan rockygerung',
 'batuan langka tasbih jokowi hadiah dari habib luthfi seharga mercy',
 'di era jokowi ekonomi indonesia semakin baik indonesia maju jokowi lagi jokowi menang total debat',
 'bagi sumatera selatan asian games berdampak pada ekonomi langsung diprediksi mencapai triliun indonesia maju jokowi hebat',
 'negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat terus masyarakatnya ngeluh karena negara ngutang setiap negara itu pasti ngutang utang bisa dibayar kalo negara dapet penghasilan penghasilan negara itu ya dari pajak']

In [82]:
display(tweets_cleaned[338]) # 266,
print('--'*50)
display(tweet_spell_corrected[338])

'aku memang setuju macro ekonominya jokowi jago bgt meski mungkin itu memang rancangan yg sudah ditulis sm ahli yg merancang pembangunan indonesia ya tapi jokowi memang memahami dg baik'

----------------------------------------------------------------------------------------------------


'aku memang setuju macro ekonominya jokowi jago banget meski mungkin itu memang rancangan yang sudah ditulis sama ahli yang merancang pembangunan indonesia ya tapi jokowi memang memahami dengan baik'

# Stopword Removal

In [13]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [83]:
# stopwords_id = ['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', 'oh', 'ok', 'seharusnya', 'sebetulnya', 'setiap', 'setidaknya', 'sesuatu', 'pasti', 'saja', 'toh', 'walau', 'tolong', 'tentu', 'amat', 'apalagi', 'bagaimanapun']

custom_stopwords = ['yah', 'ah', 'yang', 'ke', 'dan']

In [84]:
def remove_stopwords_indo(text):
    # Kode untuk menghapus stopwords
    text_lower = text.lower()
    stop_words = set(custom_stopwords)
    tokens = word_tokenize(text_lower)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

In [85]:
tweet_no_stopwords = []

for idx in range(len(tweet_spell_corrected)):
    cleaned = remove_stopwords_indo(tweet_spell_corrected[idx])
    tweet_no_stopwords.append(cleaned)

display(tweet_spell_corrected[248])
display(tweet_no_stopwords[248])

'dipikirnya pengembangan esport itu main doang aplikasi yang ada di smartphone itu banyak yang mulainya dari main doang terus jadi aplikasi yang banyak manfaatnya kata siapa pengangguran digaji banyak baca buka wawasan jangan mpermalukan generasi z'

'dipikirnya pengembangan esport itu main doang aplikasi ada di smartphone itu banyak mulainya dari main doang terus jadi aplikasi banyak manfaatnya kata siapa pengangguran digaji banyak baca buka wawasan jangan mpermalukan generasi z'

# Texts Augmentation

In [ ]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.2 MB/s eta 0:00:00


In [17]:
import random
import sentencepiece
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import wordnet
from transformers import T5ForConditionalGeneration, T5Tokenizer

C:\Users\LENOVO\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...


True

In [19]:
# Load the T5 model and tokenizer for style transfer and text paraphrasing
model_style_transfer = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer_style_transfer = T5Tokenizer.from_pretrained("t5-small")

C:\Users\LENOVO\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LENOVO\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected

In [20]:
# Function for synonym replacement using WordNet
def synonym_replacement(text, probability=0.1):
    words = word_tokenize(text)
    augmented_words = []

    for word in words:
        if random.uniform(0, 1) < probability:
            synonyms = wordnet.synsets(word, lang='ind')
            if synonyms:
                synonym = synonyms[0].lemmas(lang='ind')[0].name()
                augmented_words.append(synonym)
            else:
                augmented_words.append(word)
        else:
            augmented_words.append(word)

    return ' '.join(augmented_words)

def style_transfer(text, model, tokenizer):
    # Specify the desired style transfer task using a prefix
    input_text = "change the style: " + text

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate the style-transferred text
    output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1)

    # Decode and return the transformed text
    style_transferred_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return style_transferred_text

def text_paraphrasing(text, model, tokenizer):
    # Specify the text paraphrasing task using a prefix
    input_text = "paraphrase: " + text

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate the paraphrased text
    output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1)

    # Decode and return the paraphrased text
    paraphrased_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return paraphrased_text

def sentence_shuffling(text, probability=0.1):
    sentences = sent_tokenize(text)

    if random.uniform(0, 1) < probability:
        random.shuffle(sentences)
        return ' '.join(sentences)
    else:
        return text

def sentence_deletion(text, probability=0.1):
    sentences = sent_tokenize(text)

    if random.uniform(0, 1) < probability and len(sentences) > 1:
        # Delete a random sentence
        sentences.pop(random.randint(0, len(sentences) - 1))
        return ' '.join(sentences)
    else:
        return text

def word_insertion(text, probability=0.1):
    words = word_tokenize(text)

    if random.uniform(0, 1) < probability:
        # Insert a random word at a random position
        word_to_insert = "merdeka"
        position_to_insert = random.randint(0, len(words))
        words.insert(position_to_insert, word_to_insert)
        return ' '.join(words)
    else:
        return text

# Load the T5 model and tokenizer for style transfer and text paraphrasing
# model_style_transfer = T5ForConditionalGeneration.from_pretrained("t5-small")
# tokenizer_style_transfer = T5Tokenizer.from_pretrained("t5-small")

# Function to perform text augmentation using various techniques
def augment_text_list(texts, augmentation_functions, probabilities):
    augmented_texts = []

    for text in texts:
        augmented_text = text

        for func, prob in zip(augmentation_functions, probabilities):
            augmented_text = func(augmented_text, probability=prob)

        augmented_texts.append(augmented_text)

    return augmented_texts

In [86]:
tweet_augmentation_synonim = []

# Define augmentation functions and their corresponding probabilities
augmentation_functions = [synonym_replacement,]
probabilities = [0.8,]
# tweet_synm_replc = synonym_replacement(tweet_stemmed)
tweet_augmentation_synonim = augment_text_list(tweet_no_stopwords, augmentation_functions, probabilities)

In [87]:
display(tweet_no_stopwords[:10])
print('-'*50)
display(tweet_augmentation_synonim[:10])

['kata prabowo indonesia tidak dihargai bangsa asing berita ini pasti palsu buatan penguasa ya kan rockygerung',
 'batuan langka tasbih jokowi hadiah dari habib luthfi seharga mercy',
 'di era jokowi ekonomi indonesia semakin baik indonesia maju jokowi lagi jokowi menang total debat',
 'bagi sumatera selatan asian games berdampak pada ekonomi langsung diprediksi mencapai triliun indonesia maju jokowi hebat',
 'negara kita ngutang buat bngun infrastruktur udah dipake masyarakat terus masyarakatnya ngeluh karena negara ngutang setiap negara itu pasti ngutang utang bisa dibayar kalo negara dapet penghasilan penghasilan negara itu ya dari pajak',
 'bisikin bapak jokowi cuma mikirin perputaran duit di golongan tertentu esports hanya sebagian kecil dari sekian besar penentu pertumbuhan perekonomian indonesia bukan hal fundamental betul dikatakan prabowo walau dengan bahas sederhana tapi mrpkan aspek fundamental ekonomi',
 'masa tenang masih ngoceh ajattp jokowi harga mati',
 'uasdifitnah kej

--------------------------------------------------


['bercakap prabowo Indonesia tidak dihargai bangsa asing berita ini jelas imitatif buatan pemerintah bahwasanya kan rockygerung',
 'batuan jarang rosario jokowi bakat dari habib luthfi seharga mercy',
 'atas era jokowi ekonomi Indonesia semakin baik Indonesia ke_depan jokowi lagi jokowi larang total beranggar_lidah',
 'bagi Sumatera selatan asian games berdampak atas ekonomi langsung diprediksi berhasil triliun Indonesia maju jokowi hebat',
 'keadaan America ngutang berbuat bngun asas udah dipake masyarakat terus masyarakatnya ngeluh karena keadaan ngutang masing-masing keadaan itu jelas ngutang budi berupaya dibayar kalo keadaan dapet penghasilan penghasilan keadaan itu bahwasanya dari bergadai',
 'bisikin bapak jokowi cuma mikirin kisaran duit di blok pasti esports cuma beberapa kecil dari sekian besar penentu kemajuan perekonomian Indonesia belum barang asas betul dikatakan prabowo meskipun dengan bahas baru meskipun mrpkan aspek asas ekonomi',
 'kala anteng baru ngoceh ajattp jokow

In [88]:
display(tweet_no_stopwords[10])
print('-'*50)
display(tweet_augmentation_synonim[10])

'pilih orang baik pilih jelas islamnya pilih baju putih pilih pemimpin itu sudah jelas banyak pencapaiannya nah ini dia pencapaian ekonomi selama pemerintahan jokowi coblos yuk kerja pasti'

--------------------------------------------------


'ambil benda_hidup baik ambil jelas islamnya pilih baju beruban ambil ketua itu pernah jelas banyak pencapaiannya nah ini DIA kejayaan ekonomi selama pemerintahan jokowi coblos yuk bekerja jelas'

In [89]:
tweet_augmentation_synonim

['bercakap prabowo Indonesia tidak dihargai bangsa asing berita ini jelas imitatif buatan pemerintah bahwasanya kan rockygerung',
 'batuan jarang rosario jokowi bakat dari habib luthfi seharga mercy',
 'atas era jokowi ekonomi Indonesia semakin baik Indonesia ke_depan jokowi lagi jokowi larang total beranggar_lidah',
 'bagi Sumatera selatan asian games berdampak atas ekonomi langsung diprediksi berhasil triliun Indonesia maju jokowi hebat',
 'keadaan America ngutang berbuat bngun asas udah dipake masyarakat terus masyarakatnya ngeluh karena keadaan ngutang masing-masing keadaan itu jelas ngutang budi berupaya dibayar kalo keadaan dapet penghasilan penghasilan keadaan itu bahwasanya dari bergadai',
 'bisikin bapak jokowi cuma mikirin kisaran duit di blok pasti esports cuma beberapa kecil dari sekian besar penentu kemajuan perekonomian Indonesia belum barang asas betul dikatakan prabowo meskipun dengan bahas baru meskipun mrpkan aspek asas ekonomi',
 'kala anteng baru ngoceh ajattp jokow

In [104]:
def style_transfer(text, model, tokenizer):
    # Tokenize the input text without adding any prefix
    input_ids = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)

    # Generate the style-transferred text
    output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1)

    # Decode and return the transformed text
    style_transferred_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return style_transferred_text

In [105]:
tweet_augmentation = []

for idx in range(len(tweet_augmentation_synonim)):
    style_changed = style_transfer(tweet_augmentation_synonim[idx], model_style_transfer, tokenizer_style_transfer )
    tweet_augmentation.append(style_changed)

    

In [106]:
display(tweet_augmentation_synonim[10])
print('-'*50)
display(tweet_augmentation[10])

'ambil benda_hidup baik ambil jelas islamnya pilih baju beruban ambil ketua itu pernah jelas banyak pencapaiannya nah ini DIA kejayaan ekonomi selama pemerintahan jokowi coblos yuk bekerja jelas'

--------------------------------------------------


'baik ambil jelas islamnya pilih baju beruban ambil benda_hidup baik ambil jelas islamnya pilih baju beruban ambil benda_hidup baik ambil jelas islamnya pilih baju beruban ambil benda_hidup baik ambil jelas islamnya pilih baju beruban ambil'

In [107]:
tweet_augmentation

['bercakap prabowo Indonesia tidak dihargai bangsa asing berita ini jelas imitatif buatan pemerintah bahwasanya kan rockygerung kan rockygerung kan rockygerung bercakap prabowo Indonesia tidak dihargai bangsa asing berita ini jelas imitatif buatan pemerintah bah',
 'batuan jarang rosario jokowi bakat dari habib luthfi seharga mercy',
 'ke_depan jokowi lagi jokowi larang total beranggar_lidah',
 'berdampak atas ekonomi langsung diprediksi berhasil triliun Indonesia maju jokowi hebat berdampak atas ekonomi langsung diprediksi berhasil triliun Indonesia maju jokowi hebat berdampak atas ekonomi langsung diprediksi berhasil triliun Indonesia maju jokowi hebat',
 'ngutang budi berupaya dibayar kalo keadaan dapet penghasilan penghasilan keadaan itu bahwasanya dari bergadai bergadai berupaya dibayar kalo keadaan dapet penghasilan penghasilan keadaan itu bahwasanya dari bergada',
 'kecil dari sekian besar penentu kemajuan perekonomian Indonesia belum barang asas betul dikatakan prabowo meskipun

In [110]:
tweet_augmented_cleaned = []

  # Looping setiap kalimat
for sentence in tweet_augmentation:

    # sentence = re.sub(r'\b(\w+)\s+\1\b', r'\1', sentence) # Remove reapeating char
    sentence = sentence.replace('the style:', '')

    sentence = re.sub(r'[^a-zA-Z0-9\s]', ' ', sentence) # remove punctuation and special character


    sentence = re.sub(r"\s+", " ", sentence) # remove double space

    sentence = sentence.lower() # case folding

    sentence = sentence.strip()

    tweet_augmented_cleaned.append(sentence)

In [111]:
tweet_augmented_cleaned

['bercakap prabowo indonesia tidak dihargai bangsa asing berita ini jelas imitatif buatan pemerintah bahwasanya kan rockygerung kan rockygerung kan rockygerung bercakap prabowo indonesia tidak dihargai bangsa asing berita ini jelas imitatif buatan pemerintah bah',
 'batuan jarang rosario jokowi bakat dari habib luthfi seharga mercy',
 'ke depan jokowi lagi jokowi larang total beranggar lidah',
 'berdampak atas ekonomi langsung diprediksi berhasil triliun indonesia maju jokowi hebat berdampak atas ekonomi langsung diprediksi berhasil triliun indonesia maju jokowi hebat berdampak atas ekonomi langsung diprediksi berhasil triliun indonesia maju jokowi hebat',
 'ngutang budi berupaya dibayar kalo keadaan dapet penghasilan penghasilan keadaan itu bahwasanya dari bergadai bergadai berupaya dibayar kalo keadaan dapet penghasilan penghasilan keadaan itu bahwasanya dari bergada',
 'kecil dari sekian besar penentu kemajuan perekonomian indonesia belum barang asas betul dikatakan prabowo meskipun

In [112]:
empty_strings = [text for text in tweet_augmented_cleaned if text == '']
empty_strings

['', '', '', '', '', '', '', '']

In [113]:
tweet_augmented2 = pd.DataFrame(tweet_augmented_cleaned, columns=['tweet'])
tweet_augmented2

,tweet
0,bercakap prabowo indonesia tidak dihargai bang...
1,batuan jarang rosario jokowi bakat dari habib ...
2,ke depan jokowi lagi jokowi larang total beran...
3,berdampak atas ekonomi langsung diprediksi ber...
4,ngutang budi berupaya dibayar kalo keadaan dap...
...,...
1810,asal jemah bangga mengedepankan harga alas ban...
1811,harus ceramah atas aman indonesiaku perekonomi...
1812,bersama bapak jokowi ayo bekerja bersama bapak...
1813,bantu majukan perekonomian bangsa bangsa bersa...


In [33]:
tweet_augmented = pd.DataFrame(tweet_augmented_cleaned, columns=['tweet'])
tweet_augmented

,tweet
0,kata prabowo indonesia tidak dihargai bangsa a...
1,batuan jarang rosario jokowi bakat dari habib ...
2,atas era jokowi ekonomi indonesia semakin baik
3,bagi sumatera selatan asian games berdampak at...
4,negara kita ngutang buat bngun asas udah dipak...
...,...
1810,negarawan asal jemah bangga mengedepankan harg...
1811,harus ceramah di damai indonesiaku perekonomia...
1812,biar bangkit bangsa dengan mendukung perekonom...
1813,bantu majukan perekonomian bangsa bersama bapa...


In [114]:
df2 = pd.read_csv(f'{datasets_path}/tweet.csv')
df2

,Unnamed: 0,sentimen,tweet
0,0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...
1,1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib..."
2,2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik...."
3,3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p..."
4,4,negatif,Negara kita ngutang buat bngun infrastruktur y...
...,...,...,...
1810,1810,netral,Negarawan sejati sll bangga dan mengedepankan ...
1811,1811,netral,1. HRS ceramah di Damai Indonesiaku 2. Perekon...
1812,1812,netral,Mari bangun bangsa dgn mendukung perekonomian ...
1813,1813,netral,Bantu majukan perekonomian bangsa bersama Pak ...


In [116]:
df2['tweet'] = tweet_augmented2['tweet']
df2

,sentimen,tweet
0,negatif,bercakap prabowo indonesia tidak dihargai bang...
1,netral,batuan jarang rosario jokowi bakat dari habib ...
2,netral,ke depan jokowi lagi jokowi larang total beran...
3,positif,berdampak atas ekonomi langsung diprediksi ber...
4,negatif,ngutang budi berupaya dibayar kalo keadaan dap...
...,...,...
1810,netral,asal jemah bangga mengedepankan harga alas ban...
1811,netral,harus ceramah atas aman indonesiaku perekonomi...
1812,netral,bersama bapak jokowi ayo bekerja bersama bapak...
1813,netral,bantu majukan perekonomian bangsa bangsa bersa...


In [34]:
df['tweet'] = tweet_augmented['tweet']
df

,sentimen,tweet
0,negatif,kata prabowo indonesia tidak dihargai bangsa a...
1,netral,batuan jarang rosario jokowi bakat dari habib ...
2,netral,atas era jokowi ekonomi indonesia semakin baik
3,positif,bagi sumatera selatan asian games berdampak at...
4,negatif,negara kita ngutang buat bngun asas udah dipak...
...,...,...
1810,netral,negarawan asal jemah bangga mengedepankan harg...
1811,netral,harus ceramah di damai indonesiaku perekonomia...
1812,netral,biar bangkit bangsa dengan mendukung perekonom...
1813,netral,bantu majukan perekonomian bangsa bersama bapa...


In [115]:
del df2[df2.columns[0]]

In [117]:
df_augmented2 = df2
df_augmented2

,sentimen,tweet
0,negatif,bercakap prabowo indonesia tidak dihargai bang...
1,netral,batuan jarang rosario jokowi bakat dari habib ...
2,netral,ke depan jokowi lagi jokowi larang total beran...
3,positif,berdampak atas ekonomi langsung diprediksi ber...
4,negatif,ngutang budi berupaya dibayar kalo keadaan dap...
...,...,...
1810,netral,asal jemah bangga mengedepankan harga alas ban...
1811,netral,harus ceramah atas aman indonesiaku perekonomi...
1812,netral,bersama bapak jokowi ayo bekerja bersama bapak...
1813,netral,bantu majukan perekonomian bangsa bangsa bersa...


In [35]:
df_augmented = df
df_augmented

,sentimen,tweet
0,negatif,kata prabowo indonesia tidak dihargai bangsa a...
1,netral,batuan jarang rosario jokowi bakat dari habib ...
2,netral,atas era jokowi ekonomi indonesia semakin baik
3,positif,bagi sumatera selatan asian games berdampak at...
4,negatif,negara kita ngutang buat bngun asas udah dipak...
...,...,...
1810,netral,negarawan asal jemah bangga mengedepankan harg...
1811,netral,harus ceramah di damai indonesiaku perekonomia...
1812,netral,biar bangkit bangsa dengan mendukung perekonom...
1813,netral,bantu majukan perekonomian bangsa bersama bapa...


In [36]:
df_real = pd.read_csv(f'{datasets_path}/tweet.csv')
df_real

,Unnamed: 0,sentimen,tweet
0,0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...
1,1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib..."
2,2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik...."
3,3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p..."
4,4,negatif,Negara kita ngutang buat bngun infrastruktur y...
...,...,...,...
1810,1810,netral,Negarawan sejati sll bangga dan mengedepankan ...
1811,1811,netral,1. HRS ceramah di Damai Indonesiaku 2. Perekon...
1812,1812,netral,Mari bangun bangsa dgn mendukung perekonomian ...
1813,1813,netral,Bantu majukan perekonomian bangsa bersama Pak ...


In [37]:
del df_real[df_real.columns[0]]

In [38]:
df_real

,sentimen,tweet
0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...
1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib..."
2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik...."
3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p..."
4,negatif,Negara kita ngutang buat bngun infrastruktur y...
...,...,...
1810,netral,Negarawan sejati sll bangga dan mengedepankan ...
1811,netral,1. HRS ceramah di Damai Indonesiaku 2. Perekon...
1812,netral,Mari bangun bangsa dgn mendukung perekonomian ...
1813,netral,Bantu majukan perekonomian bangsa bersama Pak ...


In [39]:
df_real['tweet'] = tweet_no_stopwords
df_real

,sentimen,tweet
0,negatif,kata prabowo indonesia tidak dihargai bangsa a...
1,netral,batuan langka tasbih jokowi hadiah dari habib ...
2,netral,di era jokowi ekonomi indonesia semakin baik
3,positif,bagi sumatera selatan asian games berdampak pa...
4,negatif,negara kita ngutang buat bngun infrastruktur u...
...,...,...
1810,netral,negarawan sejati selalu bangga mengedepankan h...
1811,netral,harus ceramah di damai indonesiaku perekonomia...
1812,netral,mari bangun bangsa dengan mendukung perekonomi...
1813,netral,bantu majukan perekonomian bangsa bersama bapa...


# Merge Dataset

In [40]:
data_merge = pd.concat([df_real, df_augmented], ignore_index=False)
data_merge

,sentimen,tweet
0,negatif,kata prabowo indonesia tidak dihargai bangsa a...
1,netral,batuan langka tasbih jokowi hadiah dari habib ...
2,netral,di era jokowi ekonomi indonesia semakin baik
3,positif,bagi sumatera selatan asian games berdampak pa...
4,negatif,negara kita ngutang buat bngun infrastruktur u...
...,...,...
1810,netral,negarawan asal jemah bangga mengedepankan harg...
1811,netral,harus ceramah di damai indonesiaku perekonomia...
1812,netral,biar bangkit bangsa dengan mendukung perekonom...
1813,netral,bantu majukan perekonomian bangsa bersama bapa...


In [41]:
data_merge['sentimen'].value_counts()

sentimen
positif    1224
netral     1214
negatif    1192
Name: count, dtype: int64

In [42]:
from sklearn.utils import shuffle

In [43]:
data_augmntd = shuffle(data_merge)
data_augmntd

,sentimen,tweet
1051,negatif,sayang sekali bapak jokowi tidak membahas ekon...
576,positif,selama ini presiden jokowi kunker keliling ind...
795,netral,prabowo salahkan presiden berasal jokowi ahy b...
1022,positif,terlihat jelas dalam debat terakhir jokowi san...
1568,positif,prabowo sandi komit gajinya tidak akan diambil...
...,...,...
1751,netral,kalau harga pangan rendah supaya konsumen seja...
1082,positif,jokowi yakin optimis jokowi bisa menjawab kebi...
534,positif,dia juga ingin merasakan peluh keringat juga d...
775,netral,jokowi mantabkan ekonomi prabowo salahkan masa...


In [44]:
data_augmntd.to_csv(f'{datasets_path}/tweet_aug_shuffle.csv')

In [127]:
dataset_augmented = pd.DataFrame(columns=['sentimen', 'tweet'])

# for  real_sentimen, real_tweet, augmntd_sentimen, augmntd_tweet in zip(df_real['sentimen'], df_real['tweet'], df_augmented['sentimen'], df_augmented['tweet']):
#   dataset_augmented = dataset_augmented.append({'sentimen': real_sentimen, 'tweet': real_tweet}, ignore_index=True)
#   dataset_augmented = dataset_augmented.append({'sentimen': augmntd_sentimen, 'tweet': augmntd_tweet}, ignore_index=True)
# dataset_augmented = pd.concat([dataset_augmented, df_real, df_augmented], axis=0, ignore_index=True)
for real_sentimen, real_tweet, augmntd_sentimen, augmntd_tweet, augmntd_sentimen2, augmntd_tweet2 in zip(df_real['sentimen'], df_real['tweet'], df_augmented['sentimen'], df_augmented['tweet'], df_augmented2['sentimen'], df_augmented2['tweet']):
    dataset_augmented = pd.concat([dataset_augmented, pd.DataFrame({'sentimen': [real_sentimen, augmntd_sentimen, augmntd_sentimen2], 'tweet': [real_tweet, augmntd_tweet, augmntd_tweet2]})], ignore_index=True)


dataset_augmented

,sentimen,tweet
0,negatif,kata prabowo indonesia tidak dihargai bangsa a...
1,negatif,kata prabowo indonesia tidak dihargai bangsa a...
2,negatif,bercakap prabowo indonesia tidak dihargai bang...
3,netral,batuan langka tasbih jokowi hadiah dari habib ...
4,netral,batuan jarang rosario jokowi bakat dari habib ...
...,...,...
5440,netral,bantu majukan perekonomian bangsa bersama bapa...
5441,netral,bantu majukan perekonomian bangsa bangsa bersa...
5442,netral,bapak jokowi mengubah cara pandang ekonomi kin...
5443,netral,bapak jokowi malih cara pandang ekonomi dulu k...


In [128]:
dataset_augmented.sentimen.value_counts()

sentimen
positif    1836
netral     1821
negatif    1788
Name: count, dtype: int64

In [124]:
from collections import Counter

In [129]:
datasets_counts = Counter(dataset_augmented['tweet'])

# Display counts
for text, count in datasets_counts.items():
    if count > 1:
        print(f"Text: {text}, Count: {count}")

Text: kata prabowo indonesia tidak dihargai bangsa asing berita ini pasti palsu buatan penguasa ya kan rockygerung, Count: 2
Text: batuan jarang rosario jokowi bakat dari habib luthfi seharga mercy, Count: 2
Text: presiden jokowi raja salman sepakat tingkatkan kerjasama ekonomi via apahabarcom, Count: 2
Text: calar hidupmu bisanya cuma harga mahal turunkan solusi benteng prabowo bahlull, Count: 2
Text: winter, Count: 3
Text: jokowi naikkan tax ratio drastis bisa guncang ekonomi, Count: 5
Text: steuerquote drastis berupaya guncang ekonomi drastis berupaya guncang ekonomi drastis berupaya guncang ekonomi b jokowi naikkan tax ratio drastis berupaya guncang ekonomi b jokowi naikkan tax ratio drastis berupaya guncang ekonomi b, Count: 3
Text: pemerintah jokowi juga menyebut masa depan ekonomi indonesia di tangan anakanak muda kreatif, Count: 2
Text: pentingnya sisi industri kreatif inovatif di era industri global membuat jokowi tergerak untuk mengubah badan ekonomi kreatif bekraf menjadi se

In [130]:
dataset_augmented.to_csv(f'{datasets_path}/tweet_aug_4.csv')

In [131]:
empty_strings = [text for text in dataset_augmented['tweet'] if text == '']
empty_strings

['', '', '', '', '', '', '', '', '', '', '', '']